In [1]:
import numpy as np
import h5py

In [3]:
file = h5py.File('/scratch/ns4486/numerical-relativity-interpolation/Proca_fiducial_scaled_cropped.hdf5', 'r')

input_np = np.array(file['Train']['input'])
target_np = np.array(file['Train']['target'])

In [16]:
x_mean = np.mean(input_np, axis=tuple(range(input_np.ndim-3)), keepdims=True)
x_std = np.std(input_np, axis=tuple(range(input_np.ndim-3)), keepdims=True)
x_mean.shape

(1, 1, 72, 72, 72)

In [32]:
test = np.divide(input_np - x_mean, x_std, out=np.zeros_like(input_np), where=x_std!=0)
test.shape

(407, 2, 72, 72, 72)

In [24]:
x_std.min(), x_std.max(), x_std.mean()

(0.0, 0.9376054100302021, 0.010276341051667375)

In [31]:
for i in range(72):
    for j in range(72):
        for k in range(72):
            if x_std[0, 0, i, j, k] == 0:
                print(i, j, k)
                print(x_mean[0, 0, i, j, k])

35 35 35
0.0
35 35 36
0.0
35 36 35
0.0
35 36 36
0.0
36 35 35
0.0
36 35 36
0.0
36 36 35
0.0
36 36 36
0.0


In [29]:
x_std[x_std == 0]

array([0., 0., 0., 0., 0., 0., 0., 0.])

In [25]:
np.argmin(x_std)

183995

In [27]:
def argmin_lastNaxes(A, N):
    s = A.shape
    new_shp = s[:-N] + (np.prod(s[-N:]),)
    max_idx = A.reshape(new_shp).argmin(-1)
    return np.unravel_index(max_idx, s[-N:])

In [28]:
argmin_lastNaxes(x_std, 3)

(array([[35]]), array([[35]]), array([[35]]))

In [17]:
test = (input_np - x_mean)/(x_std) 
test.shape

<ipython-input-17-171be96ac4d2>:1: RuntimeWarning: invalid value encountered in true_divide
  test = (input_np - x_mean)/(x_std)


(407, 2, 72, 72, 72)

In [1]:
import torch
from torch import nn
import torch.nn.functional as F


class ConvBlock(nn.Module):
    
    def __init__(self, depth, num_layers, non_linearity):
        super(ConvBlock, self).__init__()
        
        self.depth = depth
        self.non_linearity = non_linearity
        
        self.in_channels = 16 * (2 ** int((num_layers - depth) / 2))
        self.out_channels = 16 * (2 ** int((num_layers - depth - 1) / 2)) if depth != num_layers else 1
        
#         print(self.in_channels, self.out_channels)
        
        # reduces input shape by 4
        self.conv = nn.Conv3d(
            in_channels = self.in_channels,
            out_channels = self.out_channels,
            kernel_size = (5, 5, 5),
            stride = (1, 1, 1),
            padding = (1, 1, 1)
        )
        
        self.bn = nn.BatchNorm3d(self.out_channels, affine = True)
        
        self.maxpool = nn.MaxPool3d(
            kernel_size = (3, 3, 3),
            stride = (1, 1, 1)
        )
    
    def forward(self, x):
        x = self.maxpool(
                self.non_linearity(
                    self.bn(
                        self.conv(x)
                    )
                )
            )
        
        return x


class ConvInverseBlock(nn.Module):
    
    def __init__(self, depth, original_input_size, non_linearity):
        super(ConvInverseBlock, self).__init__()
        
        self.depth = depth
        self.non_linearity = non_linearity
        
        self.in_channels = 16 * (2 ** int((depth - 2) / 2)) if depth != 1 else 1
        self.out_channels = 16 * (2 ** int((depth - 1) / 2))
        
        upsample_size = original_input_size + 4 * (depth - 1) + 6
        
#         print(upsample_size)
        
        self.upsample = nn.Upsample(
            size = upsample_size,
            mode = 'nearest'
        )
        
        self.conv_r = nn.Conv3d(
            in_channels = self.in_channels,
            out_channels = self.out_channels,
            kernel_size = (5, 5, 5),
            stride = (1, 1, 1),
            padding = (1, 1, 1)
        )
        
        self.bn = nn.BatchNorm3d(self.out_channels, affine = True)

    
    def forward(self, x):
        x = self.non_linearity(
                self.bn(
                    self.conv_r(
                        self.upsample(x)
                    )
                )
            )
        
        return x



class Encoder(nn.Module):
    
    def __init__(self, num_layers, original_input_size, non_linearity):
        super(Encoder, self).__init__()
        
        self.non_linearity = non_linearity
        
        modules = []
            
        for i in range(1, num_layers + 1):
            modules.append(ConvInverseBlock(i, original_input_size, non_linearity))
        
        self.conv = nn.Sequential(*modules)
    
    def forward(self, x):
        x = self.conv(x)
        
        return x


class Decoder(nn.Module):
    def __init__(self, num_layers, non_linearity):
        super(Decoder, self).__init__()
        
        self.non_linearity = non_linearity
                
        modules = []
        
        for i in range(1, num_layers + 1):
            modules.append(ConvBlock(i, num_layers, self.non_linearity))
        
        self.conv_inv = nn.Sequential(*modules)
    
    def forward(self, x):
        x = self.conv_inv(x)
        
        return x

In [7]:
import torch.nn.functional as F

model = nn.Sequential(
    Encoder(5, 72, F.relu), 
    Decoder(5, F.relu), 
)

In [8]:
model

Sequential(
  (0): Encoder(
    (conv): Sequential(
      (0): ConvInverseBlock(
        (upsample): Upsample(size=78, mode=nearest)
        (conv_r): Conv3d(1, 16, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(1, 1, 1))
        (bn): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ConvInverseBlock(
        (upsample): Upsample(size=82, mode=nearest)
        (conv_r): Conv3d(16, 16, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(1, 1, 1))
        (bn): BatchNorm3d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): ConvInverseBlock(
        (upsample): Upsample(size=86, mode=nearest)
        (conv_r): Conv3d(16, 32, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(1, 1, 1))
        (bn): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (3): ConvInverseBlock(
        (upsample): Upsample(size=90, mode=nearest)
        (conv_r): Conv3d(32, 32, kernel_

In [9]:
from torchsummary import summary

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
summary(model.to(device), (1, 72, 72, 72))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
          Upsample-1        [-1, 1, 78, 78, 78]               0
            Conv3d-2       [-1, 16, 76, 76, 76]           2,016
       BatchNorm3d-3       [-1, 16, 76, 76, 76]              32
  ConvInverseBlock-4       [-1, 16, 76, 76, 76]               0
          Upsample-5       [-1, 16, 82, 82, 82]               0
            Conv3d-6       [-1, 16, 80, 80, 80]          32,016
       BatchNorm3d-7       [-1, 16, 80, 80, 80]              32
  ConvInverseBlock-8       [-1, 16, 80, 80, 80]               0
          Upsample-9       [-1, 16, 86, 86, 86]               0
           Conv3d-10       [-1, 32, 84, 84, 84]          64,032
      BatchNorm3d-11       [-1, 32, 84, 84, 84]              64
 ConvInverseBlock-12       [-1, 32, 84, 84, 84]               0
         Upsample-13       [-1, 32, 90, 90, 90]               0
           Conv3d-14       [-1, 32, 88,